In [3]:
# 假设白名单保存在 filtered_barcodes.txt 中，每行一个条形码
with open(r'D:\空间转录组理论知识与工具\used_barcodes.txt', 'r') as f:
    filtered_barcodes = [line.strip() for line in f.readlines()]

In [4]:
import pandas as pd
# 读取表达矩阵文件
# 注意：需要正确处理第一行的特殊格式
expr_matrix = pd.read_csv(r'D:\空间转录组理论知识与工具\Puck_200127_15.digital_expression.txt', sep='\t', header=0)
# 第一列是基因名，后续列是细胞条形码
# 筛选出白名单中的细胞
filtered_expr = expr_matrix[['GENE'] + filtered_barcodes]

# 保存筛选后的表达矩阵
filtered_expr.to_csv('filtered_expression_matrix.csv', index=False)

In [ ]:
# 查看前5行
filtered_expr_head = filtered_expr.head()
print("筛选后的表达矩阵前5行：")
print(filtered_expr_head)

In [8]:
# 读取空间坐标文件
spatial_data = pd.read_csv(r'D:\空间转录组理论知识与工具\Puck_200127_15_bead_locations.csv')  # 需要你先整理这个文件

# 筛选出白名单中的细胞
filtered_spatial = spatial_data[spatial_data['barcode'].isin(filtered_barcodes)]

# 保存筛选后的空间数据
filtered_spatial.to_csv('filtered_spatial_coordinates.csv', index=False)

In [9]:
# 创建简洁的元数据表
metadata = pd.DataFrame({
    'Cell_ID': filtered_barcodes,
    'Sample': 'Pack_200127_15'  # 所有细胞来自同一个样本
})

# 如果需要加入空间坐标
if not filtered_spatial.empty:
    metadata = metadata.merge(
        filtered_spatial[['barcode', 'xcoord', 'ycoord']],
        left_on='Cell_ID',
        right_on='barcode'
    ).drop('barcode', axis=1)

metadata.to_csv('final_cell_metadata.csv', index=False)

In [ ]:
input_file1='/content/final_cell_metadata.csv'
input_file2='/content/filtered_expression_matrix.csv'
run_options = Values()
run_options.NN_dir = './output/Slide-seqV2_NN'
run_options.GNN_dir = './output/Slide-seqV2_GNN'
run_options.NT_dir = './output/Slide-seqV2_NT'
run_options.meta_input = input_file1
run_options.expr_input = input_file2
run_options.n_neighbors = 50
run_options.device = 'cuda'  # 将 'cuda' 改为 'cpu'
run_options.epochs = 1000
run_options.hidden_feats = 4
run_options.n_gcn_layers = 2
run_options.k = 6
run_options.modularity_loss_weight = 0.3
run_options.purity_loss_weight = 300.0
run_options.regularization_loss_weight = 0.1
run_options.beta = 0.03
run_options.trajectory_construct = 'BF'
run_options.seed = 42           # ← 缺少的
run_options.lr = 0.03           # ← 缺少的（学习率）
run_options.patience = 100      # ← 缺少的
run_options.min_delta = 0.001   # ← 缺少的
run_options.min_epochs = 50     # ← 缺少的

In [ ]:
run_options.n_local = 20  # ← 缺少的关键参数！用于定义生态位大小的最近邻数
run_options.batch_size = 0 # use full batch training怎么是0？

In [ ]:
vis_options = Values()
vis_options.NN_dir = run_options.NN_dir
vis_options.GNN_dir = run_options.GNN_dir
vis_options.NT_dir = run_options.NT_dir
vis_options.output = None
vis_options.reverse = True

In [ ]:
niche_network_construct(options=run_options) 